In [1]:
!pip install metatrader5

In [2]:
import MetaTrader5 as mt5
import time
from os.path import exists
import operator
from datetime import datetime, timedelta
import pandas as pd
from numpy import nan
import calendar

In [6]:
class SymbolUtil():

    @staticmethod
    def get_ticker_from_date(date: str, symbol: str) -> str:
        """Gets the specific ticker from the given date.

        Args:
            date (str): date (Example: 20210531)
            symbol (str): symbol (Example: WDO or WIN)

        Returns:
            ticker (str): complete symbol (For example, WINJ21)
        """

        if "WIN" not in symbol.upper() and "WDO" not in symbol.upper():
            return symbol

        return SymbolUtil.apply_win_rule(date) \
            if "WIN" in symbol else SymbolUtil.apply_wdo_rule(date)

    @staticmethod
    def apply_wdo_rule(input_date: str) -> str:
        """Recovers the specific ticker for WDO from the given date

        Args:
            input_date (str): date in the yyyymmdd format (Example: 20210531)

        Returns:
            str: specific ticker (For example: WDOM21)
        """
        input_month = input_date[4:6]
        input_year = input_date[2:4]
        input_day = input_date[6:]
        last_day = SymbolUtil.last_business_day_in_month(
            int(input_year), int(input_month))
        wdo_rules = {
            '01': 'WDOG',
            '02': 'WDOH',
            '03': 'WDOJ',
            '04': 'WDOK',
            '05': 'WDOM',
            '06': 'WDON',
            '07': 'WDOQ',
            '08': 'WDOU',
            '09': 'WDOV',
            '10': 'WDOX',
            '11': 'WDOZ',
            '12': 'WDOF',
            '13': 'WDOG'
        }
        if int(input_day) >= last_day:
            input_month = str(int(input_month) + 1).zfill(2)
        if input_month == "12":
            input_year = str(int(input_year) + 1)
        return wdo_rules[input_month] + input_year

    @staticmethod
    def apply_win_rule(input_date: str) -> str:
        """Recovers the specific ticker for WIN from the given date

        Args:
            input_date (str): date in the yyyymmdd format (Example: 20210531)

        Returns:
            str: specific ticker (For example: WINM21)
        """
        win_rules = {
            '01': 'WING',
            '03': 'WINJ',
            '05': 'WINM',
            '07': 'WINQ',
            '09': 'WINV',
            '11': 'WINZ'
        }
        input_month = input_date[4:6]
        input_year = input_date[:4]
        input_day = input_date[6:]
        year_win_last_letters = input_date[2:4]
        fifteen_day = datetime(int(input_year), int(input_month), 15)
        day_week_15 = fifteen_day.weekday()
        next_rule = SymbolUtil.get_next_wednesday(day_week_15)
        win_rule_is = None
        if int(input_month) % 2 == 0:
            if int(input_day) >= next_rule:
                month_rule = str(int(input_month) + 1)
                if int(month_rule) > 12:
                    month_rule = "1"
                month_rule = month_rule.rjust(2, '0')
                win_rule_is = win_rules[month_rule]
            else:
                month_rule = str(int(input_month) - 1)
                month_rule = month_rule.rjust(2, '0')
                win_rule_is = win_rules[month_rule]
        else:
            win_rule_is = win_rules[input_month]
        if win_rule_is == "WING" and input_month == "12":
            year_win_last_letters = str(int(year_win_last_letters) + 1)
            return win_rule_is + year_win_last_letters
        else:
            return win_rule_is + year_win_last_letters

    @staticmethod
    def last_business_day_in_month(year: int, month: int) -> int:
        """Gets the last business day from the given period of time

        Args:
            year (int): year (Example: 2021)
            month (int): month (Example: 10)

        Returns:
            int: last business day
        """
        return max(calendar.monthcalendar(year, month)[-1:][0][:5])

    @staticmethod
    def get_next_wednesday(day15: int) -> int:
        """Gets the Wednesday closest to the 15th of the month

        Args:
            day15 (int): represents the day of the week of 15th of the month

        Returns:
            int: day of the month closest to the 15th
        """
        if day15 == 6:
            return 15 + 3
        elif day15 < 2:
            return (2 - day15) + 15
        elif day15 > 2 and day15 < 6:
            return 15 - (day15 - 2)
        else:
            return 15

In [ ]:
SYMBOL_PREFIX = "WIN"
SYMBOL = SymbolUtil.get_ticker_from_date(datetime.now().strftime("%Y%m%d"), SYMBOL_PREFIX)
MAX_DEPTH = 10

start_time = '09:05:00'
end_time = '17:55:00'

folder_path = 'data2/'

time_delta = timedelta(milliseconds=10)
batch_size = 1000

print(SYMBOL)

WINM25


In [9]:
# establish connection to the MetaTrader 5 terminal
if not mt5.initialize():
    print("initialize() failed, error code =",mt5.last_error())
   # shut down connection to the MetaTrader 5 terminal
    mt5.shutdown()
    quit()
    
# Testing the display of the SYMBOL in MarketWatch
selected=mt5.symbol_select(SYMBOL,True)
if not selected:
    print("Failed to select ", SYMBOL)
    mt5.shutdown()
    
# subscribe to market depth updates for SYMBOL (Depth of Market)
if mt5.market_book_add(SYMBOL):
    # get the market depth data
    items = mt5.market_book_get(SYMBOL)
    # display the entire market depth 'as is' in a single string
    print(items)
    # now display each order separately for more clarity
    if items:
        for it in items:
            # order content
            print(it._asdict())
else:
    print(f"mt5.market_book_add('{SYMBOL}') failed, error code = {mt5.last_error()}")

()


In [10]:
def get_timed_book(symbol: str, max_depth: int = 10) -> dict:
    book_data = mt5.market_book_get(symbol)
    tick_data = mt5.symbol_info_tick(symbol)
    
    bids = sorted((book_info for book_info in book_data if book_info.type == 2), key= operator.attrgetter("price"), reverse=True)[:max_depth]
    asks = sorted((book_info for book_info in book_data if book_info.type == 1), key= operator.attrgetter("price"), reverse=False)[:max_depth]
     
    obj = dict(
            time = tick_data.time_msc,
            last = tick_data.last,
            mid =  (tick_data.bid + tick_data.ask) / 2,
            spread = (tick_data.ask - tick_data.bid)
        )
    
    len_bids = len(bids) 
    len_asks = len(asks)
    
    i: int = 0

    for i in range(max_depth):
        if len_asks > i:
            obj['ask_price_'+str(i)] = asks[i].price
            obj['ask_volume_'+str(i)] = asks[i].volume_dbl
        else:
            obj['ask_price_'+str(i)] = nan
            obj['ask_volume_'+str(i)] = nan
            
        if len_bids > i:
            obj['bid_price_'+str(i)] = bids[i].price
            obj['bid_volume_'+str(i)] = bids[i].volume_dbl
        else:
            obj['bid_price_'+str(i)] = nan
            obj['bid_volume_'+str(i)] = nan
            
    return obj
 
print(get_timed_book(SYMBOL))
%timeit book_data = get_timed_book(SYMBOL)

{'time': 1748626612752, 'last': 138155.0, 'mid': 138152.5, 'spread': 5.0, 'ask_price_0': 138155.0, 'ask_volume_0': 40.0, 'bid_price_0': 138150.0, 'bid_volume_0': 126.0, 'ask_price_1': 138160.0, 'ask_volume_1': 115.0, 'bid_price_1': 138145.0, 'bid_volume_1': 209.0, 'ask_price_2': 138165.0, 'ask_volume_2': 165.0, 'bid_price_2': 138140.0, 'bid_volume_2': 481.0, 'ask_price_3': 138170.0, 'ask_volume_3': 284.0, 'bid_price_3': 138135.0, 'bid_volume_3': 376.0, 'ask_price_4': 138175.0, 'ask_volume_4': 247.0, 'bid_price_4': 138130.0, 'bid_volume_4': 433.0, 'ask_price_5': 138180.0, 'ask_volume_5': 533.0, 'bid_price_5': 138125.0, 'bid_volume_5': 259.0, 'ask_price_6': 138185.0, 'ask_volume_6': 615.0, 'bid_price_6': 138120.0, 'bid_volume_6': 474.0, 'ask_price_7': 138190.0, 'ask_volume_7': 646.0, 'bid_price_7': 138115.0, 'bid_volume_7': 420.0, 'ask_price_8': 138195.0, 'ask_volume_8': 457.0, 'bid_price_8': 138110.0, 'bid_volume_8': 298.0, 'ask_price_9': 138200.0, 'ask_volume_9': 543.0, 'bid_price_9': 

In [11]:
start = datetime.strptime(start_time, '%H:%M:%S').time()
end = datetime.strptime(end_time, '%H:%M:%S').time()

data = []
i=0
    
# market not opened yet or market is already closed
while ( (datetime.now().time() < start) or (datetime.now().time() > end) ): 
        time.sleep(1)
        
file_path = folder_path + SYMBOL + datetime.now().date().strftime("_%Y-%m-%d") + ".csv"

old_data = get_timed_book(SYMBOL, MAX_DEPTH)

if not exists(file_path):
    df = pd.DataFrame([old_data])
    df.to_csv(file_path, index=False, mode='w', header=True)
    print(f"New file created ! {file_path}")
else:
    print("File was already created! No new file created !")

time_now = datetime.now()
last_saved_time = time_now    
    
while start < time_now.time() < end:
    time_now = datetime.now()
    
    if last_saved_time + time_delta < time_now:
        last_saved_time = time_now
        i+=1
        
        new_data = get_timed_book(SYMBOL, MAX_DEPTH)
        if new_data['time'] != old_data['time']:
            data.append(new_data)
        old_data = new_data
        
        if i % batch_size == 0:
            df = pd.DataFrame(data)
            df.to_csv(file_path, index=False, mode='a', header=False)
            i = 0
            data = []
            print("Saved on ", time_now)
            
mt5.market_book_release(SYMBOL)

# shut down connection to the MetaTrader 5 terminal
mt5.shutdown()

New file created ! data2/WINM25_2025-05-30.csv
Saved on  2025-05-30 17:37:14.361505
Saved on  2025-05-30 17:37:30.467239
Saved on  2025-05-30 17:37:46.052116
Saved on  2025-05-30 17:38:01.285356
Saved on  2025-05-30 17:38:16.118127
Saved on  2025-05-30 17:38:32.386045
Saved on  2025-05-30 17:38:48.736193
Saved on  2025-05-30 17:39:05.051704
Saved on  2025-05-30 17:39:21.587988
Saved on  2025-05-30 17:39:37.523726
Saved on  2025-05-30 17:39:52.725002
Saved on  2025-05-30 17:40:08.805155
Saved on  2025-05-30 17:40:25.244433
Saved on  2025-05-30 17:40:41.529905
Saved on  2025-05-30 17:40:58.083593


KeyboardInterrupt: 

In [12]:
mt5.market_book_release(SYMBOL)

# shut down connection to the MetaTrader 5 terminal
mt5.shutdown()

True

In [24]:
df_final = pd.read_csv(file_path)
df_final.to_parquet(file_path.replace('.csv', '.parquet'), index=False)
print(f"Data saved to {file_path.replace('.csv', '.parquet')}")

# Deleting the CSV file after conversion to Parquet
import os
if os.path.exists(file_path):
    os.remove(file_path)
    print(f"Deleted the CSV file: {file_path}")

Data saved to data2/WINM25_2025-05-30.parquet
Deleted the CSV file: data2/WINM25_2025-05-30.csv
